In [119]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy



if (not os.path.isfile('data/spy-train.csv')):
    start = datetime.datetime(2015, 2, 1)
    end = datetime.datetime.now()
    df_train = data.DataReader("SPY", 'yahoo', start, end)
    df_train.to_csv("data/spy-train.csv")

df_train = pd.read_csv('data/spy-train.csv', index_col='Date', parse_dates=True)
latestDate = datetime.datetime.now() - datetime.timedelta(days=1)
if (df_train.index[-1] < latestDate):
    start = df_train.index[-1] + datetime.timedelta(days=1)
    new_data = data.DataReader("SPY", 'yahoo', start, latestDate)
    df_train.append(new_data)

df_train.to_csv("data/spy-train.csv")

df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)

df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)
#print(df_train)

In [88]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()

df_train = df_train.assign(pma10=pma10)
df_train = df_train.assign(pma20=pma20)
df_train = df_train.assign(pma50=pma50)
df_train = df_train.assign(pma200=pma200)

In [116]:
diffma10 = (df_train['Close']/df_train['pma10'] -1)*100
diffma20 = (df_train['Close']/df_train['pma20'] -1)*100
diffma50 = (df_train['Close']/df_train['pma50'] -1)*100
diffma200 = (df_train['Close']/df_train['pma200'] -1)*100

df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

In [111]:
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

df_train = df_train.assign(pma10tan=pma10tan)
df_train = df_train.assign(pma20tan=pma20tan)
df_train = df_train.assign(pma50tan=pma50tan)
df_train = df_train.assign(pma200tan=pma200tan)


In [113]:
hh = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hl = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 

df_train = df_train.assign(hh=hh)
df_train = df_train.assign(hl=hl)

hh10 = hh.rolling(window=10, center=False).mean()
hl10 = hl.rolling(window=10, center=False).mean()

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

In [117]:
#print(hh10)